In [1]:
import os
from argparse import ArgumentParser




In [7]:
import argparse

from distutils.util import strtobool




def get_parser():
    parser = argparse.ArgumentParser()
    
    # Discriminator
    parser.add_argument('--D_num_outcomes', help='No of discriminator outcomes', 
                        default=32)    
    parser.add_argument('--D_filters', nargs='+', type=int, 
                        default=[3, 32, 64],
                        help="Examples: -D_filters 3 32 64")
    parser.add_argument('--D_act_name', 
                        help='Discriminator activation name Swish or LeakyReLU',
                        default='Swish')
    parser.add_argument('--D_use_adaptive_reparam', dest='D_use_adaptive_reparam', 
                        default=True, type=lambda x: bool(strtobool(x)))
    parser.add_argument('--D_use_spectral_norm', dest='D_use_spectral_norm', 
                        default=False, type=lambda x: bool(strtobool(x)))
    
    # Stylist
    parser.add_argument('--backbone', default='vgg11')
    
    # Generator
    parser.add_argument('--dlatent_size', type=int, default=128)
    parser.add_argument('--in_channel', type=int, default=3)
    parser.add_argument('--out_channel',type=int,help='kernel size',default=32)
    
    
    #networks hyper parameters:
    parser.add_argument('--nfc', type=int, default=32)
    parser.add_argument('--min_nfc', type=int, default=32)
    parser.add_argument('--ker_size',type=int,help='kernel size',default=3)
    parser.add_argument('--num_layer',type=int,help='number of layers',default=5)
    parser.add_argument('--stride',help='stride',default=1)
    parser.add_argument('--padd_size',type=int,help='net pad size',default=0)#math.floor(opt.ker_size/2)
    
    #parser.add_argument('--mode', help='task to be done', default='train')
    #workspace:
    parser.add_argument('--not_cuda', action='store_true', help='disables cuda', default=0)
    
    #load, input, save configurations:
    parser.add_argument('--netG', default='', help="path to netG (to continue training)")
    parser.add_argument('--netD', default='', help="path to netD (to continue training)")
    parser.add_argument('--manualSeed', type=int, help='manual seed')
    parser.add_argument('--nc_z',type=int,help='noise # channels',default=3)
    parser.add_argument('--nc_im',type=int,help='image # channels',default=3)
    parser.add_argument('--out',help='output folder',default='Output')
        
    #pyramid parameters:
    parser.add_argument('--scale_factor',type=float,help='pyramid scale factor',default=0.75)#pow(0.5,1/6))
    parser.add_argument('--noise_amp',type=float,help='addative noise cont weight',default=0.1)
    parser.add_argument('--min_size',type=int,help='image minimal size at the coarser scale',default=25)
    parser.add_argument('--max_size', type=int,help='image minimal size at the coarser scale', default=250)

    #optimization hyper parameters:
    parser.add_argument('--niter', type=int, default=2000, help='number of epochs to train per scale')
    parser.add_argument('--gamma',type=float,help='scheduler gamma',default=0.1)
    parser.add_argument('--lr_g', type=float, default=0.0005, help='learning rate, default=0.0005')
    parser.add_argument('--lr_d', type=float, default=0.0005, help='learning rate, default=0.0005')
    parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
    parser.add_argument('--Gsteps',type=int, help='Generator inner steps',default=3)
    parser.add_argument('--Dsteps',type=int, help='Discriminator inner steps',default=3)
    parser.add_argument('--lambda_grad',type=float, help='gradient penelty weight',default=0.1)
    parser.add_argument('--alpha',type=float, help='reconstruction loss weight',default=10)

    
    return parser

parser  = get_parser()
config = parser.parse_args(args=[])
config

Namespace(D_act_name='Swish', D_filters=[3, 32, 64], D_num_outcomes=32, D_use_adaptive_reparam=True, D_use_spectral_norm=False, Dsteps=3, Gsteps=3, alpha=10, backbone='vgg11', beta1=0.5, dlatent_size=128, gamma=0.1, in_channel=3, ker_size=3, lambda_grad=0.1, lr_d=0.0005, lr_g=0.0005, manualSeed=None, max_size=250, min_nfc=32, min_size=25, nc_im=3, nc_z=3, netD='', netG='', nfc=32, niter=2000, noise_amp=0.1, not_cuda=0, num_layer=5, out='Output', out_channel=32, padd_size=0, scale_factor=0.75, stride=1)

In [1]:
from src.config import get_parser

parser  = get_parser()
config = parser.parse_args(args=[])
config

Namespace(D_act_name='Swish', D_filters=[3, 32, 64], D_num_outcomes=32, D_use_adaptive_reparam=True, D_use_spectral_norm=False, Dsteps=3, Gsteps=3, alpha=10, backbone='vgg11', beta1=0.5, dlatent_size=128, gamma=0.1, in_channel=3, ker_size=3, lambda_grad=0.1, lr_d=0.0005, lr_g=0.0005, manualSeed=None, max_size=250, min_nfc=32, min_size=25, nc_im=3, nc_z=3, netD='', netG='', nfc=32, niter=2000, noise_amp=0.1, not_cuda=0, num_layer=5, out='Output', out_channel=32, padd_size=0, scale_factor=0.75, stride=1)

In [2]:
from src.models.discriminator import Discriminator

D = Discriminator(config)
print(D)
import torch
D(torch.rand(1, 3, 32, 32)).shape

Discriminator(
  (model): Sequential(
    (b0): DiscriminatorBlock(
      (conv_res): Conv2d(3, 32, kernel_size=(1, 1), stride=(2, 2))
      (net): Sequential(
        (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act1): Swish()
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act2): Swish()
      )
      (downsample): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (b1): DiscriminatorBlock(
      (conv_res): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      (net): Sequential(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act1): Swish()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act2): Swish()
      )
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Linear(in_features=64, out_features=32, bias=True)
  (reparam):

torch.Size([1, 32])

In [3]:
from src.models.generator import Generator

G = Generator(config)
print(G)
img = torch.rand(1, 3, 128, 128)
style = torch.rand(1, config.dlatent_size)
G(img, style).shape

Generator(
  (head): ModConvLayer(
    (conv): EqualizedModConv2d(3, 32, 3, upsample=False, downsample=False)
  )
  (body): Sequential(
    (block1): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block2): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (block3): ConvBlock(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (LeakyRelu): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (tail): Sequential(
    (0): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): Tanh()
  )
)
to

torch.Size([1, 3, 120, 120])

In [4]:
config.backbone

'vgg11'

In [5]:
from src.models.stylist import Stylist

S = Stylist(config)
print(S)
S(torch.rand(1, 3, 64, 64)).shape

Stylist(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

torch.Size([1, 128])